In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

import numpy as np
from matplotlib import pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
from config import db_password

import datetime
from datetime import date
from datetime import time

import re

## Read Kaggle data from matches.csv

Import the table `matches.csv` from Kaggle data on major league soccer (MLS).

In [133]:
# Read .csv from FinalProject/Resources
file_dir = "../Resources/matches.csv"    

# Create a pandas DataFrame 
game_df = pd.read_csv(file_dir, low_memory=False, skipinitialspace = True)

# Show all columns
pd.set_option("display.max_columns", None)

game_df

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [134]:
results = game_df['home_score'] - game_df['away_score']
game_df['result'] = results
game_df.head(2)

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation,result
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [135]:
results

0       2
1       0
2      -1
3       1
4       2
       ..
6693   -4
6694    2
6695    0
6696    0
6697    0
Length: 6698, dtype: int64

In [136]:
results[2]

-1

In [137]:
outcome = []
for i in range(len(results)):
    if results[i] > 0:
        outcome.append('win')
    elif results[i] < 0:
        outcome.append('loss')
    else:
        outcome.append('tie')
    
game_df['Outcome'] = outcome
game_df.head(2)

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation,result,Outcome
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [26]:
# Check column headers
pd.set_option('display.max_columns', None)
print(game_df.columns.values)


['id' 'home' 'away' 'date' 'year' 'time (utc)' 'attendance' 'venue'
 'league' 'part_of_competition' 'game_status' 'shootout' 'home_score'
 'away_score' 'referee' 'home_goal_minutes' 'home_goal_scorers'
 'away_goal_minutes' 'away_goal_scorers' 'home_starting_1_num'
 'home_starting_1' 'home_starting_2_num' 'home_starting_2'
 'home_starting_3_num' 'home_starting_3' 'home_starting_4_num'
 'home_starting_4' 'home_starting_5_num' 'home_starting_5'
 'home_starting_6_num' 'home_starting_6' 'home_starting_7_num'
 'home_starting_7' 'home_starting_8_num' 'home_starting_8'
 'home_starting_9_num' 'home_starting_9' 'home_starting_10_num'
 'home_starting_10' 'home_starting_11_num' 'home_starting_11'
 'home_bench_1_num' 'home_bench_1' 'home_bench_1_minute'
 'home_bench_2_num' 'home_bench_2' 'home_bench_2_minute'
 'home_bench_3_num' 'home_bench_3' 'home_bench_3_minute'
 'home_bench_4_num' 'home_bench_4' 'home_bench_4_minute'
 'home_bench_5_num' 'home_bench_5' 'home_bench_5_minute'
 'home_bench_6_num' '

In [138]:
# Keep only those columns that information can be known before a game starts.
game_df = game_df[['home', 'away', 'date', 'year', 'time (utc)', 'attendance', 'venue', 'Outcome']]
game_df

,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,win
1,Dallas,Colorado,6/15/1996,1996,NaN,"9,704",Cotton Bowl,tie
2,Colorado,D.C. United,8/29/1996,1996,NaN,"6,368",Mile High Stadium,loss
3,LA Galaxy,New England,8/8/1996,1996,NaN,"10,251",Rose Bowl,win
4,New England,D.C. United,7/20/1996,1996,NaN,"18,347",Foxboro Stadium,win
...,...,...,...,...,...,...,...,...
6693,Vancouver Whitecaps,Real Salt Lake,"Wednesday, July 7",2021,2:00,"12,009","Rio Tinto Stadium, Sandy, UT",loss
6694,LA Galaxy,FC Dallas,"Wednesday, July 7",2021,2:30,"16,453","Dignity Health Sports Park, Carson",win
6695,Nashville SC,Atlanta United FC,"Thursday, July 8",2021,0:30,"22,913","Nissan Stadium, Nashville, Nashville, TN",tie
6696,New York Red Bulls,Philadelphia Union,"Thursday, July 8",2021,0:30,"10,013","Red Bull Arena, Harrison",tie


In [139]:
# Check null values
game_df.isnull().sum()

home             0
away             0
date             0
year             0
time (utc)     972
attendance    2355
venue          467
Outcome          0
dtype: int64

In [140]:
# Drop null values
game_df = game_df.dropna()
game_df

,home,away,date,year,time (utc),attendance,venue,Outcome
754,Colorado,LA Galaxy,10/24/1999,1999,13:00 (14:00),"6,542",Mile High Stadium,loss
2236,Columbus Crew SC,Toronto FC,"Saturday, March 29",2008,20:00,"13,843","MAPFRE Stadium, Columbus",win
2237,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",2008,22:00,"20,272","Rice-Eccles Stadium, Salt Lake City",tie
2238,Sporting Kansas City,DC United,"Saturday, March 29",2008,23:00,"10,385",Community America Ballpark,win
2239,New England Revolution,Houston Dynamo,"Saturday, March 29",2008,23:30,"11,116",Gillette Stadium,win
...,...,...,...,...,...,...,...,...
6693,Vancouver Whitecaps,Real Salt Lake,"Wednesday, July 7",2021,2:00,"12,009","Rio Tinto Stadium, Sandy, UT",loss
6694,LA Galaxy,FC Dallas,"Wednesday, July 7",2021,2:30,"16,453","Dignity Health Sports Park, Carson",win
6695,Nashville SC,Atlanta United FC,"Thursday, July 8",2021,0:30,"22,913","Nissan Stadium, Nashville, Nashville, TN",tie
6696,New York Red Bulls,Philadelphia Union,"Thursday, July 8",2021,0:30,"10,013","Red Bull Arena, Harrison",tie


In [144]:
game_df['home'].unique()

array(['Colorado Rapids', 'Columbus Crew SC', 'Real Salt Lake',
       'Sporting Kansas City', 'New England Revolution', 'FC Dallas',
       'Chicago Fire FC', 'LA Galaxy', 'DC United', 'New York Red Bulls',
       'Chivas USA', 'Houston Dynamo', 'San Jose Earthquakes',
       'Toronto FC', 'Seattle Sounders FC', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Philadelphia Union',
       'Orlando City SC', 'New York City FC', 'Atlanta United FC',
       'Minnesota United FC', 'LAFC', 'FC Cincinnati',
       'Houston Dynamo FC', 'Nashville SC', 'Inter Miami CF',
       'Columbus Crew', 'Austin FC'], dtype=object)

In [145]:
game_df['away'].unique()

array(['LA Galaxy', 'Toronto FC', 'Chicago Fire FC', 'DC United',
       'Houston Dynamo', 'Chivas USA', 'New England Revolution',
       'San Jose Earthquakes', 'Colorado Rapids', 'Columbus Crew SC',
       'Real Salt Lake', 'FC Dallas', 'New York Red Bulls',
       'Sporting Kansas City', 'Seattle Sounders FC',
       'Philadelphia Union', 'Montreal Impact', 'Portland Timbers',
       'Vancouver Whitecaps', 'New York City FC', 'Orlando City SC',
       'Minnesota United FC', 'Atlanta United FC', 'LAFC',
       'FC Cincinnati', 'Inter Miami CF', 'Houston Dynamo FC',
       'Nashville SC', 'Austin FC', 'CF Montréal', 'Columbus Crew'],
      dtype=object)

In [148]:
game_df['home'] = game_df['home'].replace('Colorado', "Colorado Rapids")
game_df['away'] = game_df['away'].replace('Colorado', "Colorado Rapids")

game_df['home'] = game_df['home'].replace('Columbus Crew SC', "Columbus Crew")
game_df['away'] = game_df['away'].replace('Columbus Crew SC', "Columbus Crews")

game_df['home'] = game_df['home'].replace('Houston Dynamo', "Houston Dynamo FC")
game_df['away'] = game_df['away'].replace('Houston Dynamo', "Houston Dynamo FC")

game_df.head()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,home,away,date,year,time (utc),attendance,venue,Outcome
754,Colorado Rapids,LA Galaxy,10/24/1999,1999,13:00 (14:00),"6,542",Mile High Stadium,loss
2236,Columbus Crew,Toronto FC,"Saturday, March 29",2008,20:00,"13,843","MAPFRE Stadium, Columbus",win
2237,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",2008,22:00,"20,272","Rice-Eccles Stadium, Salt Lake City",tie
2238,Sporting Kansas City,DC United,"Saturday, March 29",2008,23:00,"10,385",Community America Ballpark,win
2239,New England Revolution,Houston Dynamo FC,"Saturday, March 29",2008,23:30,"11,116",Gillette Stadium,win


In [149]:
game_df['home'].unique()

array(['Colorado Rapids', 'Columbus Crew', 'Real Salt Lake',
       'Sporting Kansas City', 'New England Revolution', 'FC Dallas',
       'Chicago Fire FC', 'LA Galaxy', 'DC United', 'New York Red Bulls',
       'Chivas USA', 'Houston Dynamo FC', 'San Jose Earthquakes',
       'Toronto FC', 'Seattle Sounders FC', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Philadelphia Union',
       'Orlando City SC', 'New York City FC', 'Atlanta United FC',
       'Minnesota United FC', 'LAFC', 'FC Cincinnati', 'Nashville SC',
       'Inter Miami CF', 'Austin FC'], dtype=object)

In [150]:
# Check unique entries
game_df['time (utc)'].unique()

array(['13:00 (14:00)', '20:00', '22:00', '23:00', '23:30', '1:30',
       '19:00', '0:00', '2:30', '21:00', '20:30', '0:30', '1:00', '16:30',
       '19:30', '2:00', '16:00', '3:00', '22:30', '3:30', '17:00',
       '18:00', '18:30', '21:30', '17:30', '4:00', '21:55', '0:10',
       '1:44', '2:15', '1:15', '3:35', '19:55', '17:55', '0:55', '1:27',
       '2:40', '20:55', '1:47', '0:15', '1:38', '1:36', '1:52', '23:15',
       '0:02'], dtype=object)

In [151]:
# Check 'time' column
game_df['time (utc)'] = game_df['time (utc)'].replace('13:00 (14:00)', "14:00")
#game_df['time (utc)'].drop(game_df['time (utc)'] == '13:00 (14:00)')
game_df['time (utc)'] 

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


754     14:00
2236    20:00
2237    22:00
2238    23:00
2239    23:30
        ...  
6693     2:00
6694     2:30
6695     0:30
6696     0:30
6697    23:30
Name: time (utc), Length: 3559, dtype: object

In [152]:
# Replace the ',' from attendance values.
game_df['attendance'] = game_df['attendance'].str.replace(",", "").astype("float")
game_df.head(2)

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,home,away,date,year,time (utc),attendance,venue,Outcome
754,Colorado Rapids,LA Galaxy,10/24/1999,1999,14:00,6542.0,Mile High Stadium,loss
2236,Columbus Crew,Toronto FC,"Saturday, March 29",2008,20:00,13843.0,"MAPFRE Stadium, Columbus",win


In [153]:
game_df['date'].unique

<bound method Series.unique of 754             10/24/1999
2236    Saturday, March 29
2237    Saturday, March 29
2238    Saturday, March 29
2239    Saturday, March 29
               ...        
6693     Wednesday, July 7
6694     Wednesday, July 7
6695      Thursday, July 8
6696      Thursday, July 8
6697        Friday, July 9
Name: date, Length: 3559, dtype: object>

In [154]:
game_df['date'] = game_df['date'].astype('datetime64')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-03-29 00:00:00

In [79]:
import re

baaar = game_df['date']
dates = []
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for day in days:
    if re.search(baaar, days):
        dates.append('day')
    else:
        dates.append('')
dates

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [80]:
game_df['time (utc)'].loc[754]

'14:00'

In [155]:
game_df = game_df.drop(["date", "time (utc)"], axis='columns')
game_df.head(2)

,home,away,year,attendance,venue,Outcome
754,Colorado Rapids,LA Galaxy,1999,6542.0,Mile High Stadium,loss
2236,Columbus Crew,Toronto FC,2008,13843.0,"MAPFRE Stadium, Columbus",win


In [156]:
game_df.isnull().sum()


home          0
away          0
year          0
attendance    0
venue         0
Outcome       0
dtype: int64

In [157]:
game_df.nunique()

home            28
away            30
year            14
attendance    2803
venue           74
Outcome          3
dtype: int64

In [158]:
game_df.dtypes

home           object
away           object
year            int64
attendance    float64
venue          object
Outcome        object
dtype: object

It looks like there are 4 columns with **categorical** values having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [159]:
# Generate our categorical variable list
game_cat = game_df.dtypes[game_df.dtypes == "object"].index.tolist()
game_cat

['home', 'away', 'venue', 'Outcome']

In [160]:
# Check the number of unique values in each column
game_df[game_cat].nunique()

home       28
away       30
venue      74
Outcome     3
dtype: int64

In [161]:
game_df['home'].unique()

array(['Colorado Rapids', 'Columbus Crew', 'Real Salt Lake',
       'Sporting Kansas City', 'New England Revolution', 'FC Dallas',
       'Chicago Fire FC', 'LA Galaxy', 'DC United', 'New York Red Bulls',
       'Chivas USA', 'Houston Dynamo FC', 'San Jose Earthquakes',
       'Toronto FC', 'Seattle Sounders FC', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Philadelphia Union',
       'Orlando City SC', 'New York City FC', 'Atlanta United FC',
       'Minnesota United FC', 'LAFC', 'FC Cincinnati', 'Nashville SC',
       'Inter Miami CF', 'Austin FC'], dtype=object)

Before going to encode these columns using Scikit-learn's OneHotEncoder module, we need to make sure that if the categorical variables require **bucketing**. Since we want to predict outcome for each individual `home` team, we need to see the relation of all individual `home` team, `venue` and `game_status`, we choose not to to any binning/bucketing, and we're ready to use **OneHotEncoder**.

In [162]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(game_df[game_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [163]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(game_cat)
encode_df.head()

,home_Atlanta United FC,home_Austin FC,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus Crew,away_Columbus Crews,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,venue_Allianz Field,"venue_Allianz Field, Minnesota",venue_Audi Field,"venue_Audi Field, Washington DC",venue_BBVA Stadium,"venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver",venue_BMO Field,venue_Banc of California Stadium,"venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,"venue_Camping World Stadium, Orlando","venue_CenturyLink Field, Seattle","venue_Children's Mercy Park, Kansas City","venue_Children's Mercy Park, Kansas City, KS",venue_Citi Field Stadium,venue_Community America Ballpark,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dick's Sporting Goods Park, Denver","venue_Dignity Health Sports Park, Carson","venue_Dignity Health Sports Park, Los Angeles",venue_Earthquakes Stadium,venue_Exploria Stadium,"venue_Exploria Stadium, Orlando, Florida",venue_FedExField,"venue_Giants Stadium, East Rutherford",venue_Gillette Stadium,"venue_Historic Crew Stadium, Columbus, OH",venue_Inter Miami CF Stadium,venue_Levi's Stadium,"venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_McAfee Coliseum,venue_Mercedes-Benz Stadium,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Mile High Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville","venue_Nissan Stadium, Nashville, Nashville, TN","venue_PayPal Park, San Jose",venue_Pizza Hut Park,venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland","venue_Q2 Stadium, Austin","venue_Qwest Field, Seattle","venue_RFK Memorial Stadium, Washington, D.C.","venue_RFK Stadium, Washington, D.C.",venue_Red Bull Arena,"venue_Red Bull Arena, Harrison","venue_Rice-Eccles Stadium, Salt Lake City",venue_Rio Tinto Stadium,"venue_Rio Tinto Stadium, Sandy, UT","venue_Rio Tinto Stadium, Utah","venue_Robertson Stadium, Houston",venue_Rogers Centere,venue_SeatGeek Stadium,"venue_Soldier Field, Chicago",venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,venue_StubHub Center,"venue_Subaru Park, Chester","venue_Subaru Park, Philadelphia",venue_TCF Bank Stadium,"venue_TQL Stadium, Cincinnati, OH",venue_Toyota Field,venue_Toyota Stadium,"venue_Toyota Stadium, Frisco, TX",venue_Yankee Stadium,Outcome_loss,Outcome_tie,Outcome_win
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [164]:
# Merge OneHotEncoded features and drop the originals from the updated df
game_df = game_df.merge(encode_df, left_index=True, right_index=True).drop(game_df[game_cat],1)
game_df.head()

,year,attendance,home_Atlanta United FC,home_Austin FC,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus Crew,away_Columbus Crews,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,venue_Allianz Field,"venue_Allianz Field, Minnesota",venue_Audi Field,"venue_Audi Field, Washington DC",venue_BBVA Stadium,"venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver",venue_BMO Field,venue_Banc of California Stadium,"venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,"venue_Camping World Stadium, Orlando","venue_CenturyLink Field, Seattle","venue_Children's Mercy Park, Kansas City","venue_Children's Mercy Park, Kansas City, KS",venue_Citi Field Stadium,venue_Community America Ballpark,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dick's Sporting Goods Park, Denver","venue_Dignity Health Sports Park, Carson","venue_Dignity Health Sports Park, Los Angeles",venue_Earthquakes Stadium,venue_Exploria Stadium,"venue_Exploria Stadium, Orlando, Florida",venue_FedExField,"venue_Giants Stadium, East Rutherford",venue_Gillette Stadium,"venue_Historic Crew Stadium, Columbus, OH",venue_Inter Miami CF Stadium,venue_Levi's Stadium,"venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_McAfee Coliseum,venue_Mercedes-Benz Stadium,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Mile High Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville","venue_Nissan Stadium, Nashville, Nashville, TN","venue_PayPal Park, San Jose",venue_Pizza Hut Park,venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland","venue_Q2 Stadium, Austin","venue_Qwest Field, Seattle","venue_RFK Memorial Stadium, Washington, D.C.","venue_RFK Stadium, Washington, D.C.",venue_Red Bull Arena,"venue_Red Bull Arena, Harrison","venue_Rice-Eccles Stadium, Salt Lake City",venue_Rio Tinto Stadium,"venue_Rio Tinto Stadium, Sandy, UT","venue_Rio Tinto Stadium, Utah","venue_Robertson Stadium, Houston",venue_Rogers Centere,venue_SeatGeek Stadium,"venue_Soldier Field, Chicago",venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,venue_StubHub Center,"venue_Subaru Park, Chester","venue_Subaru Park, Philadelphia",venue_TCF Bank Stadium,"venue_TQL Stadium, Cincinnati, OH",venue_Toyota Field,venue_Toyota Stadium,"venue_Toyota Stadium, Frisco, TX",venue_Yankee Stadium,Outcome_loss,Outcome_tie,Outcome_win
754,1999,6542.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the `Outcome_win` column from the rest of the input data. Also, the `Outcome_loss`, `Outcome_tie`columns have the same input for all the data, so we can drop this column.

In [165]:
# Split our preprocessed data into our features and target arrays
y = game_df["Outcome_win"].values
X = game_df.drop(["Outcome_win","Outcome_loss", "Outcome_tie" ], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [166]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **three hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the `sigmoid` activation function that will help us predict the probability that a team is winning or not.

In [167]:
len(X_train[0])

134

In [168]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  268
hidden_nodes_layer2 = 134
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 268)               36180     
_________________________________________________________________
dense_9 (Dense)              (None, 134)               36046     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1350      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 73,587
Trainable params: 73,587
Non-trainable params: 0
_________________________________________________________________


In [169]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
18/18 [==============================] - 6s 24ms/step - loss: 0.7044 - accuracy: 0.4964 0s - loss: 0.7073 - accuracy: 0.
Epoch 2/100
18/18 [==============================] - 0s 25ms/step - loss: 0.6404 - accuracy: 0.6691 0s - loss: 0.6512 - accu
Epoch 3/100
18/18 [==============================] - 0s 19ms/step - loss: 0.5917 - accuracy: 0.7000
Epoch 4/100
18/18 [==============================] - 0s 26ms/step - loss: 0.5385 - accuracy: 0.7545
Epoch 5/100
18/18 [==============================] - 1s 34ms/step - loss: 0.4890 - accuracy: 0.7818
Epoch 6/100
18/18 [==============================] - 0s 25ms/step - loss: 0.4395 - accuracy: 0.8273 0s - loss: 0.4328 - accuracy
Epoch 7/100
18/18 [==============================] - 0s 21ms/step - loss: 0.3893 - accuracy: 0.8582
Epoch 8/100
18/18 [==============================] - 0s 27ms/step - loss: 0.3574 - accuracy: 0.8564
Epoch 9/100
18/18 [==============================] - 0s 20ms/step - loss: 0.2993 - accuracy: 0.8964
Epoch 10/100


18/18 [==============================] - 1s 29ms/step - loss: 0.0296 - accuracy: 0.9855
Epoch 75/100
18/18 [==============================] - ETA: 0s - loss: 0.0274 - accuracy: 0.98 - 1s 30ms/step - loss: 0.0274 - accuracy: 0.9873
Epoch 76/100
18/18 [==============================] - 0s 23ms/step - loss: 0.0283 - accuracy: 0.9909
Epoch 77/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0268 - accuracy: 0.9891 0s - loss: 0.0238 - accuracy: 0.
Epoch 78/100
18/18 [==============================] - 0s 23ms/step - loss: 0.0270 - accuracy: 0.9873
Epoch 79/100
18/18 [==============================] - 0s 26ms/step - loss: 0.0221 - accuracy: 0.9927
Epoch 80/100
18/18 [==============================] - 0s 25ms/step - loss: 0.0332 - accuracy: 0.9855
Epoch 81/100
18/18 [==============================] - 0s 21ms/step - loss: 0.0261 - accuracy: 0.9891
Epoch 82/100
18/18 [==============================] - 0s 24ms/step - loss: 0.0268 - accuracy: 0.9836
Epoch 83/100
18/18 [=========

In [ ]:
gaussian